<a href="https://colab.research.google.com/github/dkanzariya/AI-Notebooks/blob/master/communication_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torchnet

     |████████████████████████████████| 686kB 4.8MB/s 
     |████████████████████████████████| 204kB 15.0MB/s 
  Created wheel for torchnet: filename=torchnet-0.0.4-cp36-none-any.whl size=29744 sha256=e65d04c4e44648d1b0f7362d17c30bd87cf7b62fd3f995121ede9a3956f00e0c
  Stored in directory: /root/.cache/pip/wheels/e1/03/fb/1c212c2f20905cdf97fe39022946cf16b8e66ed754a6663400
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655250 sha256=5f16457e338704a5faa7a9a956bc725fc6926b5cccc134662e536420ce93f918
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=7d2270df367e1beed3c724625700e69229ccdef4bfe9d2b43a233dab532f0361
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchnet visdom torchfile


In [3]:
import torch
from torch import nn
from tqdm import tqdm
from torchnet.engine import Engine
from torch.autograd import Variable
from torch.optim import Adam
import torchnet as tnt
import math

In [2]:
num_epochs = 100
batch_size = 256
channel_size = 4

In [ ]:
class RadioTransNets(nn.Module):
  def __init__(self,in_channel,compossed_dim):
    super(RadioTransNets,self).__init__()

    self.in_channel = in_channel

    self.encoder = nn.Sequential(
        nn.Linear(in_channel , in_channel),
        nn.ReLU(inplace = True),
        nn.Linear(in_channel,compossed_dim) 
    )
    
    self.decoder = nn.Sequential(
        nn.Linear(compossed_dim , compossed_dim),
        nn.ReLU(inplace = True),
        nn.Linear(compossed_dim ,in_channel) 
    )

  def decode_signal(self,x):
    return self.decoder(x)

  def forward(self,x):
    x = self.encoder(x)

    x = (self.in_channel ** 2) * (x/x.norm(dim = -1)[: , None])

    train_snr = 5.01187
    communication_rate = 1

    noise = Variable(torch.randn(*x.size()) / ((2* communication_rate * train_snr) ** 0.5))

    x+=noise
    x = self.decoder(x)

    return x

In [ ]:
    model = RadioTransNets(channel_size, compossed_dim=int(math.log2(channel_size)))

    train_labels = (torch.rand(10000) * channel_size).long()
    train_data = torch.sparse.torch.eye(channel_size).index_select(dim=0, index=train_labels)

    test_labels = (torch.rand(1500) * channel_size).long()
    test_data = torch.sparse.torch.eye(channel_size).index_select(dim=0, index=test_labels)

    optimizer = Adam(model.parameters())

    engine = Engine()
    meter_loss = tnt.meter.AverageValueMeter()
    meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
    confusion_meter = tnt.meter.ConfusionMeter(channel_size, normalized=True)

    loss_fn = nn.CrossEntropyLoss()

In [ ]:
def get_iterator(mode):
    data = train_data if mode else test_data
    labels = train_labels if mode else test_labels
    tensor_dataset = tnt.dataset.TensorDataset([data, labels])

    return tensor_dataset.parallel(batch_size=batch_size, num_workers=4, shuffle=mode)

def processor(sample):
        data, labels, training = sample

        data = Variable(data)
        labels = Variable(labels)

        outputs = model(data)

        loss = loss_fn(outputs, labels)

        return loss, outputs

def reset_meters():
        meter_accuracy.reset()
        meter_loss.reset()
        confusion_meter.reset()

def on_sample(state):
        state['sample'].append(state['train'])

def on_forward(state):
        meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
        confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
        meter_loss.add(state['loss'].data)

def on_start_epoch(state):
        reset_meters()
        state['iterator'] = tqdm(state['iterator'])

def on_end_epoch(state):
        print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
            state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

        reset_meters()

        engine.test(processor, get_iterator(False))

        print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
            state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

In [ ]:
engine.hooks['on_sample'] = on_sample
engine.hooks['on_forward'] = on_forward
engine.hooks['on_start_epoch'] = on_start_epoch
engine.hooks['on_end_epoch'] = on_end_epoch

engine.train(processor, get_iterator(True), maxepoch=num_epochs, optimizer=optimizer)

100%|██████████| 40/40 [00:00<00:00, 101.72it/s]

[Epoch 1] Training Loss: 2.9076 (Accuracy: 25.18%)



  0%|          | 0/40 [00:00<?, ?it/s]

[Epoch 1] Testing Loss: 2.3444 (Accuracy: 26.80%)


100%|██████████| 40/40 [00:00<00:00, 94.75it/s]

...

[Epoch 99] Testing Loss: 0.3533 (Accuracy: 74.27%)


100%|██████████| 40/40 [00:00<00:00, 115.83it/s]

[Epoch 100] Training Loss: 0.3459 (Accuracy: 74.74%)


[Epoch 100] Testing Loss: 0.3530 (Accuracy: 74.93%)


{'epoch': 100,
 'iterator': 100%|██████████| 40/40 [00:00<00:00, 85.00it/s],
 'loss': None,
 'maxepoch': 100,
 'network': <function __main__.processor>,
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     eps: 1e-08
     lr: 0.001
     weight_decay: 0
 ),
 'output': None,
 'sample': [tensor([[0., 0., 1., 0.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [1., 0., 0., 0.],
          [0., 0., 0., 1.],
          [0., 1., 0., 0.],
          [0., 0., 0., 1.],
          [0., 0., 0., 1.],
          [0., 0., 1., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 1., 0.],
          [0., 0., 1., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 1., 0.]]),
  tensor([2, 3, 2, 0, 3, 1, 3, 3, 2, 1, 2, 2, 2, 1, 2, 2]),
  True],
 't': 4000,
 'train': True}